In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/arabicdeepfakedetection5.csv')

In [ ]:
df.head()

,Unnamed: 0,Inputs,Label
0,34977,برنامج Essentials Pro 9. 0. 2. 1. 6 Portable ي...,0
1,46463,عرب بت - أعلنت اللجنة المنظمة لمهرجان التسوق ا...,0
2,36231,وكانت وزارة الاستثمار قد رفعت أسعار تلك الخدما...,0
3,71,قال وزير الخارجية البريطاني ، ويليام هيج ، الي...,0
4,42859,حجز تشيلسي فرصة التتويج بلقب دوري أبطال أوروبا...,0


In [ ]:
df=df.drop(columns='Unnamed: 0')

In [ ]:
from sklearn.model_selection import train_test_split
text = df['Inputs']
label = df['Label']


X_train, X_val, y_train, y_val = train_test_split(text, label, test_size=0.25, stratify=label)

In [ ]:
def tokenize_data(tokenizer, texts):
  """Tokenizes a list of text strings using the given BERT tokenizer.

  Args:
    tokenizer: A BERT tokenizer.
    texts: A list of text strings.

  Returns:
    A list of tokenized sentences, where each sentence is a list of token IDs.
  """

  tokenized_texts = []
  for text in texts:
    # Fix the error by removing the `return_tensors='pt'` argument.
    tokens = tokenizer(text, padding=True, truncation=True)['input_ids']
    tokenized_texts.append(tokens)

  return tokenized_texts

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 115.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification , AutoModel
from keras.layers import Embedding
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
bartmodel = AutoModel.from_pretrained("facebook/bart-base")
embedding_layer = Embedding(
    input_dim=bartmodel.config.vocab_size,
    output_dim=bartmodel.config.hidden_size,
    weights=[(bartmodel.get_input_embeddings().weight.detach())],
    trainable=False)

In [ ]:
x =tokenize_data(tokenizer, X_train)
x1 =tokenize_data(tokenizer, X_val)
import keras
import transformers
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential
x_train=pad_sequences(x)
x_val=pad_sequences(x1)

In [ ]:
from tensorflow.keras.losses import BinaryCrossentropy
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import Input
from keras.layers import Bidirectional
model= Sequential()
model.add(embedding_layer)
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs=10,steps_per_epoch=100,batch_size=32)

Epoch 1/10
100/100 [==============================] - 11s 35ms/step - loss: 0.6308 - accuracy: 0.6266
Epoch 2/10
100/100 [==============================] - 4s 38ms/step - loss: 0.5321 - accuracy: 0.7397
Epoch 3/10
100/100 [==============================] - 4s 40ms/step - loss: 0.4335 - accuracy: 0.7984
Epoch 4/10
100/100 [==============================] - 4s 37ms/step - loss: 0.4760 - accuracy: 0.7841
Epoch 5/10
100/100 [==============================] - 4s 41ms/step - loss: 0.3619 - accuracy: 0.8447
Epoch 6/10
100/100 [==============================] - 4s 39ms/step - loss: 0.3089 - accuracy: 0.8675
Epoch 7/10
100/100 [==============================] - 4s 39ms/step - loss: 0.5624 - accuracy: 0.7013
Epoch 8/10
100/100 [==============================] - 4s 40ms/step - loss: 0.3586 - accuracy: 0.8491
Epoch 9/10
100/100 [==============================] - 3s 34ms/step - loss: 0.2079 - accuracy: 0.9103
Epoch 10/10
100/100 [==============================] - 3s 34ms/step - loss: 0.1395 - accur

In [ ]:
model.evaluate(x_val,y_val,steps=100)

100/100 [==============================] - 6s 51ms/step - loss: 0.0978 - accuracy: 0.9670


[0.09777727723121643, 0.9669761657714844]

In [ ]:
text='لعلم هو مورد لا ينضب، ومن خلال الاستزادة منه يمكننا تطوير المجتمع وتحقيق التقدم الشامل في مختلف المجالات'
text_p = [text]
x2 = tokenize_data(tokenizer, text_p)
prediction_input = np.array(x2).reshape(-1)
prediction_input=pad_sequences([prediction_input],128)
output = model.predict (prediction_input)
if output > 0.5:
  predicted_class = 'human written'
else:
  predicted_class = 'machine generated'
# Print the predicted class
print('Predicted class:',predicted_class,output)

1/1 [==============================] - 0s 30ms/step
Predicted class: machine generated [[0.14095923]]


In [ ]:
model.save('bart-base.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
